In [8]:
using PowerSystems
using PowerSimulations
using HydroPowerSimulations
const PSI = PowerSimulations
using PowerSystemCaseBuilder
using Dates
using Gurobi, JuMP

In [2]:
sys = build_system(PSISystems, "modified_RTS_GMLC_DA_sys")

┌ Info: Loaded time series from storage file existing=/projects/emco4286/.julia/packages/PowerSystemCaseBuilder/dJGb8/data/serialized_system/e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855/modified_RTS_GMLC_DA_sys_time_series_storage.h5 new=/projects/emco4286/code-server/tmp/jl_F9NeoI compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /projects/emco4286/.julia/packages/InfrastructureSystems/LEg3t/src/hdf5_time_series_storage.jl:110


Property,Value
Name,
Description,
System Units Base,SYSTEM_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,501
Type,Count
ACBus,73
Arc,109
Area,3


In [11]:

template_uc = template_unit_commitment(network=NetworkModel(CopperPlatePowerModel, use_slacks = true))
solver = optimizer_with_attributes(Gurobi.Optimizer)
set_optimizer_attribute(solver, "MIPGap", 0.5)

In [14]:
models = SimulationModels(;
    decision_models = [
        DecisionModel(template_uc, sys; optimizer = solver, name = "UC"),
    ],
)

feedforward = Dict(
    "ED" => [
        SemiContinuousFeedforward(;
            component_type = ThermalStandard,
            source = OnVariable,
            affected_values = [ActivePowerVariable],
        ),
    ],
)

sequence = SimulationSequence(;
    models = models,
    ini_cond_chronology = InterProblemChronology(),
)

sim = Simulation(;
    name = "test1",
    steps = 1,
    models = models,
    sequence = sequence,
    simulation_folder = "/projects/emco4286/data/sienna_data/output/test/uc_only_gurobi",
)


Simulation
┌─────────────────┬────────────────────┐
│ Simulation Name │ test1              │
│ Build Status    │ EMPTY              │
│ Run Status      │ NOT_READY          │
│ Initial Time    │ Unset Initial Time │
│ Steps           │ 1                  │
└─────────────────┴────────────────────┘

Decision Models
┌────────────┬──────────────────┬────────┬──────────────────┐
│ Model Name │ Model Type       │ Status │ Output Directory │
├────────────┼──────────────────┼────────┼──────────────────┤
│ UC         │ GenericOpProblem │ EMPTY  │ nothing          │
└────────────┴──────────────────┴────────┴──────────────────┘

No Emulator Model Specified

Simulation Sequence
┌──────────────────────────┬──────────┐
│ Simulation Step Interval │ 24 hours │
│ Number of Problems       │ 1        │
└──────────────────────────┴──────────┘

Simulation Problems
┌────────────┬─────────┬──────────┬─────────────────────┐
│ Model Name │ Horizon │ Interval │ Executions Per Step │
├────────────┼─────────┼──────────┼─────────────────────┤
│ UC         │ 2 days  │ 1 day    │ 1                   │
└────────────┴─────────┴──────────┴─────────────────────┘

In [16]:
build!(sim)
execute!(sim; enable_progress_bar = false)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2682251
Academic license 2682251 - for non-commercial use only - registered to em___@colorado.edu
Set parameter MIPGap to value 0.5


┌ Info: 
│ ────────────────────────────────────────────────────────────────────────────────────────────
│                                                    Time                    Allocations      
│                                           ───────────────────────   ────────────────────────
│             Tot / % measured:                  10.2s / 100.0%            484MiB / 100.0%    
│ 
│ Section                           ncalls     time    %tot     avg     alloc    %tot      avg
│ ────────────────────────────────────────────────────────────────────────────────────────────
│ Build Simulation                       1    10.2s  100.0%   10.2s    484MiB  100.0%   484MiB
│   Initialize Simulation State          1    3.80s   37.2%   3.80s    147MiB   30.4%   147MiB
│   Build Decision Problems              1    1.29s   12.6%   1.29s    226MiB   46.8%   226MiB
│     Problem UC                         1    1.29s   12.6%   1.29s    226MiB   46.8%   226MiB
│       RenewableNonDispatch          

InfrastructureSystems.Simulation.RunStatusModule.RunStatus.SUCCESSFULLY_FINALIZED = 0

In [17]:
solutions = SimulationResults("/projects/emco4286/data/sienna_data/output/test/uc_only_gurobi", "test1")

┌ Info: Loading simulation results from /projects/emco4286/data/sienna_data/output/test/uc_only_gurobi/test1
└ @ PowerSimulations /projects/emco4286/.julia/packages/PowerSimulations/89s3Q/src/simulation/simulation_results.jl:87
┌ Info: checking integrity of simulation_store.h5
└ @ PowerSimulations /projects/emco4286/.julia/packages/PowerSimulations/89s3Q/src/utils/file_utils.jl:53


Decision Problem Results
┌──────────────┬─────────────────────┬────────────┬─────────────────────────┐
│ Problem Name │ Initial Time        │ Resolution │ Last Solution Timestamp │
├──────────────┼─────────────────────┼────────────┼─────────────────────────┤
│ UC           │ 2020-01-01T00:00:00 │ 1 hour     │ 2020-01-01T00:00:00     │
└──────────────┴─────────────────────┴────────────┴─────────────────────────┘

Emulator Results
┌─────────────────┬────────────┐
│ Name            │ Emulator   │
│ Resolution      │ 60 minutes │
│ Number of steps │ 24         │
└─────────────────┴────────────┘

In [18]:
results_uc = get_decision_problem_results(solutions, "UC")

ActivePowerBalance__System
ProductionCostExpression__RenewableDispatch
ProductionCostExpression__ThermalStandard
FuelConsumptionExpression__ThermalStandard
RequirementTimeSeriesParameter__VariableReserve__ReserveUp__Spin_Up_R3
ActivePowerTimeSeriesParameter__RenewableNonDispatch
RequirementTimeSeriesParameter__VariableReserve__ReserveDown__Reg_Down
ActivePowerTimeSeriesParameter__PowerLoad
RequirementTimeSeriesParameter__VariableReserve__ReserveUp__Reg_Up
RequirementTimeSeriesParameter__VariableReserve__ReserveUp__Spin_Up_R1
ActivePowerTimeSeriesParameter__RenewableDispatch


In [20]:
ren_disp = read_parameter(results_uc, "ActivePowerTimeSeriesParameter__RenewableDispatch")

DataStructures.SortedDict{Any, Any, Base.Order.ForwardOrdering} with 1 entry:
  DateTime("2020-01-01T00:00:00") => 48×30 DataFrame…

In [21]:
ren_disp[DateTime("2020-01-01T00:00:00")]

Row,DateTime,122_WIND_1,324_PV_3,312_PV_1,102_PV_1,101_PV_1,324_PV_2,313_PV_2,104_PV_1,101_PV_2,309_WIND_1,310_PV_2,113_PV_1,317_WIND_1,314_PV_1,324_PV_1,103_PV_1,303_WIND_1,314_PV_2,102_PV_2,314_PV_3,320_PV_1,101_PV_3,319_PV_1,314_PV_4,310_PV_1,215_PV_1,313_PV_1,101_PV_4,119_PV_1
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-01T00:00:00,713.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142.8,0.0,0.0,795.1,0.0,0.0,0.0,480.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01T01:00:00,712.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.1,0.0,0.0,794.4,0.0,0.0,0.0,634.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01T02:00:00,708.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145.3,0.0,0.0,773.6,0.0,0.0,0.0,487.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01T03:00:00,710.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,144.8,0.0,0.0,767.3,0.0,0.0,0.0,432.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-01-01T04:00:00,701.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137.1,0.0,0.0,752.2,0.0,0.0,0.0,407.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-01-01T05:00:00,682.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.6,0.0,0.0,719.4,0.0,0.0,0.0,440.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-01-01T06:00:00,614.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.2,0.0,0.0,655.3,0.0,0.0,0.0,377.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2020-01-01T07:00:00,517.7,36.4,52.2,30.6,29.6,36.4,62.2,29.8,30.4,47.3,38.2,99.4,594.6,47.4,44.8,29.8,199.3,46.6,30.0,63.8,47.8,30.6,179.2,46.4,38.2,127.6,62.2,28.8,27.2
9,2020-01-01T08:00:00,426.6,63.4,97.4,36.8,34.8,63.4,96.0,35.0,37.0,48.9,61.4,126.2,579.1,61.2,65.2,58.4,110.6,67.0,36.4,100.0,61.4,37.4,248.0,67.0,61.4,164.6,95.6,35.4,56.6
